In [1]:
import sys
print(sys.executable)

/usr/local/opt/python@3.11/bin/python3.11


In [1]:
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import os
import openai

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
def chat_completion(messages, tools=None, tool_choice=None, model="gpt-3.5-turbo-0613"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
        temperature=0,
    )
    return response

In [8]:
def build_message(role: str, content: str):
    return {"role": role, "content": content}

def sys_message(content: str):
    return build_message(role="system", content=content)

def user_message(content: str):
    return build_message(role="user", content=content)


In [10]:
print(chat_completion([user_message("What's the capital city of Australia")]).choices[0].message.content)

The capital city of Australia is Canberra.


## Defining function

In [11]:
func_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather for a City",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco",
                }
            },
            "required": ["city"],
        },
    }
}

In [12]:
tools = [func_get_current_weather]

In [17]:
messages = []
messages.append(sys_message("Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."))
messages.append(user_message("What's the weather like today"))

response = chat_completion(messages, tools=tools)
assistant_message = response.choices[0].message
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x10c56d190> JSON: {
  "content": "Sure, I can help you with that. Could you please tell me the name of the city you want to know the weather for?",
  "role": "assistant"
}

In [19]:
messages.append(user_message("I'm in Melbourne, Australia"))

response = chat_completion(messages, tools=tools)
assistant_message = response.choices[0].message
messages.append(assistant_message)

assistant_message

<OpenAIObject at 0x10c56cdd0> JSON: {
  "content": null,
  "role": "assistant",
  "tool_calls": [
    {
      "function": {
        "arguments": "{\n  \"city\": \"Melbourne\"\n}",
        "name": "get_current_weather"
      },
      "id": "call_6DRFtk59mt9pBIeNKMqBulKV",
      "type": "function"
    }
  ]
}

In [24]:
import requests

def get_current_weather(params):
    url = f"http://wttr.in/{params['city']}?format=3"
    response = requests.get(url)
    return response.text

In [25]:
get_current_weather({"city": "Melbourne"})

'Melbourne: ⛅️  +15°C\n'

In [27]:
import json

if assistant_message.tool_calls[0].function.name == "get_current_weather":
    params = json.loads(assistant_message.tool_calls[0].function.arguments)
    result = get_current_weather(params)
    messages.append({
        "role": "tool", 
        "tool_call_id": assistant_message.tool_calls[0].id, 
        "name": assistant_message.tool_calls[0].function.name,
        "content": result
    })

print(messages)

[{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': "What's the weather like today"}, <OpenAIObject at 0x10c56d190> JSON: {
  "content": "Sure, I can help you with that. Could you please tell me the name of the city you want to know the weather for?",
  "role": "assistant"
}, {'role': 'user', 'content': "I'm in Melbourne, Australia"}, {'role': 'user', 'content': "I'm in Melbourne, Australia"}, <OpenAIObject at 0x10c56cdd0> JSON: {
  "content": null,
  "role": "assistant",
  "tool_calls": [
    {
      "function": {
        "arguments": "{\n  \"city\": \"Melbourne\"\n}",
        "name": "get_current_weather"
      },
      "id": "call_6DRFtk59mt9pBIeNKMqBulKV",
      "type": "function"
    }
  ]
}, {'role': 'tool', 'tool_call_id': 'call_6DRFtk59mt9pBIeNKMqBulKV', 'name': 'get_current_weather', 'content': 'Melbourne: ⛅️  +15°C\n'}]


In [28]:
messages.append(user_message("Is it a good weather for camping tonight?"))
response = chat_completion(messages, tools=tools)
assistant_message = response.choices[0].message

print(assistant_message)

{
  "content": "The current weather in Melbourne is partly cloudy with a temperature of 15\u00b0C. While it may not be ideal for camping, it is still possible depending on your preferences. It's always a good idea to check the weather forecast for the evening and make sure you have appropriate camping gear.",
  "role": "assistant"
}
